## Preamble

In [ ]:
# Required packages
import numpy as np
import cv2 
from scipy.signal import savgol_filter
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from math import *
import tifffile
import warnings
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
from scipy.optimize import curve_fit
import string
from pylab import *
warnings.filterwarnings("ignore")

In [ ]:
# Plot formatting 
matplotlib.rcParams['mathtext.fontset']='cm'
matplotlib.rcParams['font.family']='STIXGeneral'
plt.rcParams['legend.fontsize']=15
plt.rcParams.update({'font.size':15}) 
plt.rcParams['axes.axisbelow']=True

## Crack orientation rose diagram of filtered lengths

#### 1. Shear Band

In [ ]:
### USING STACK HISTOGRAMS ###
########## Figure ##########
fig, ax = plt.subplots(2,3, subplot_kw={'projection':'polar'}, figsize=(17,10))
bottom, top = 0, 1
left, right = 0, 1
fig.subplots_adjust(top = top, bottom = bottom, left = left, right = right, hspace = 0.3, wspace = 0.081)
## custom ##
titles = [r'80$\mu$m > $L$', r'40$\mu$m < $L$ < 80$\mu$m', 
          r'$L$ < 40$\mu$m', r'80$\mu$m > $L$', r'40$\mu$m < $L$ < 80$\mu$m', 
           r'$L$ < 40$\mu$m']
for n, a in enumerate(ax.flat):
    #set title:
    a.set_title(titles[n])
    #set max length:
    a.set_rmax(180)
    #subplot label
    a.text(-0.15, 1.122, string.ascii_lowercase[n]+")", transform = a.transAxes, size=22, weight = 'bold')
## plot ##
## top row of plots: global thresholding
FL = [10000, 10, 5, 0]
#### ERANGA BLUE
for i in range(3):
    stackedL1 = np.zeros(36)#np.zeros(89)
    for f in range(37):
        #read csv file
        df1 = pd.read_csv('Residual_Thresholding_Fiji_output/shear_band/'+str(f)+'-nt_s2_roi.csv', delimiter=',')
        #Filter length below 10 pixels
        filt = df1[(df1['Major'] <= FL[i])&(df1['Major'] > FL[i+1])] #filter
        #make histogram  
        counts1, bins1 = np.histogram(filt['Angle'], bins=36)
        
        #stack bins   
        stackedL1 += counts1
        axis1 = bins1[:-1] 
    #print(len(stackedL1))
    two_halves1 = np.concatenate([stackedL1, stackedL1]) #concatenate
    ax[0,i].bar(np.deg2rad(np.linspace(0, 355, 72)), two_halves1, width=np.deg2rad(5),bottom=0.0,
            alpha= 0.6, color='dodgerblue',edgecolor='k') #plot

ax[0,0].set_rgrids(np.arange(0, 300, 100), angle=25)
ax[0,1].set_rgrids(np.arange(0, 300, 100), angle=25)    
ax[0,2].set_rgrids(np.arange(0, 1500, 500), angle=25)
    
#### GRIFF ORANGE
for j in range(3):
    stackedL1 = np.zeros(36)#np.zeros(89)
    for f in range(37):
        #read csv file
        df1 = pd.read_csv('Skeletonisation_Fiji_output/shear_band/'+str(f)+'-s1_lukeout_roi.csv', delimiter=',')
        #Filter length below 10 pixels
        filt = df1[(df1['Major'] <= FL[j])&(df1['Major'] > FL[j+1])] #filter
        #make histogram  
        counts1, bins1 = np.histogram(filt['Angle'], bins=36)
        
        #stack bins   
        stackedL1 += counts1
        axis1 = bins1[:-1] 

    two_halves1 = np.concatenate([stackedL1, stackedL1]) #concatenate
    ax[1,j].bar(np.deg2rad(np.linspace(0, 355, 72)), two_halves1, width=np.deg2rad(5),bottom=0.0,
            alpha= 0.6, color='darkorange',edgecolor='k') #plot
ax[1,0].set_rgrids(np.arange(0, 500, 100), angle=25)
ax[1,1].set_rgrids(np.arange(0, 500, 100), angle=25)
ax[1,2].set_rgrids(np.arange(0,  800, 200), angle=25)  
ax_flat = ax.flat
# Format radial labels in scientific notation
import matplotlib.ticker as ticker
def format_func(x, pos):
        exponent = np.floor(np.log10(x)) if x != 0 else 0
        coeff = x / 10**exponent
        return r'${0} \cdot 10^{{{1}}}$'.format(int(coeff), int(exponent)) if coeff != 1 else r'$10^{{{0}}}$'.format(int(exponent))

for a in ax_flat:
    a.yaxis.set_major_formatter(formatter)
    a.tick_params(axis='y', labelsize=16)

#### 2. Whole Image

In [ ]:
########## Figure ##########
fig, ax = plt.subplots(2,3, subplot_kw={'projection':'polar'}, figsize=(17,10))
bottom, top = 0, 1
left, right = 0, 1
fig.subplots_adjust(top = top, bottom = bottom, left = left, right = right, hspace = 0.3, wspace = 0.081)
## custom ##
titles = [r'80$\mu$m > $L$', r'40$\mu$m < $L$ < 80$\mu$m', 
          r'$L$ < 40$\mu$m', r'80$\mu$m > $L$', r'40$\mu$m < $L$ < 80$\mu$m', 
           r'$L$ < 40$\mu$m']
for n, a in enumerate(ax.flat):
    #set title:
    a.set_title(titles[n])
    #set max length:
    a.set_rmax(180)
    #subplot label
    a.text(-0.15, 1.122, string.ascii_lowercase[n]+")", transform = a.transAxes, size=22, weight = 'bold')

## top row of plots: global thresholding
FL = [10000, 10, 5, 0]
#### ERANGA BLUE
for i in range(3):
    stackedL1 = np.zeros(36)#np.zeros(89)
    for f in range(37):
        #read csv file
        df1 = pd.read_csv('Residual_Thresholding_Fiji_output/whole_image/'+str(f)+'-nt_s2.csv', delimiter=',')
        #Filter length below 10 pixels
        filt = df1[(df1['Major'] <= FL[i])&(df1['Major'] > FL[i+1])] #filter
        #make histogram  
        counts1, bins1 = np.histogram(filt['Angle'], bins=36)
        
        #stack bins   
        stackedL1 += counts1
        axis1 = bins1[:-1] 
    #print(len(stackedL1))
    two_halves1 = np.concatenate([stackedL1, stackedL1]) #concatenate
    ax[0,i].bar(np.deg2rad(np.linspace(0, 355, 72)), two_halves1, width=np.deg2rad(5),bottom=0.0,
            alpha= 0.6, color='dodgerblue',edgecolor='k') #plot
    
#### GRIFF ORANGE
for j in range(3):
    stackedL1 = np.zeros(36)#np.zeros(89)
    for f in range(37):
        #read csv file
        df1 = pd.read_csv('Skeletonisation_Fiji_output/whole_image/'+str(n)+'-s1_lukeout.csv', delimiter=',')
        #Filter length below 10 pixels
        filt = df1[(df1['Major'] <= FL[j])&(df1['Major'] > FL[j+1])] #filter
        #make histogram  
        counts1, bins1 = np.histogram(filt['Angle'], bins=36)
        
        #stack bins   
        stackedL1 += counts1
        axis1 = bins1[:-1] 

    two_halves1 = np.concatenate([stackedL1, stackedL1]) #concatenate
    ax[1,j].bar(np.deg2rad(np.linspace(0, 355, 72)), two_halves1, width=np.deg2rad(5),bottom=0.0,
            alpha= 0.6, color='darkorange',edgecolor='k') #plot

ax_flat = ax.flat
# Format radial labels in scientific notation
import matplotlib.ticker as ticker
def format_func(x, pos):
        exponent = np.floor(np.log10(x)) if x != 0 else 0
        coeff = x / 10**exponent
        return r'${0} \cdot 10^{{{1}}}$'.format(int(coeff), int(exponent)) if coeff != 1 else r'$10^{{{0}}}$'.format(int(exponent))

for a in ax_flat:
    a.set_rgrids(np.arange(0, 5100, 1000), angle=25)
    a.set_rgrids(np.arange(0, 500, 100), angle=25)
    formatter = ticker.FuncFormatter(format_func)
    a.yaxis.set_major_formatter(formatter)
    a.tick_params(axis='y', labelsize=16)

## Plotting crack orientation distribution histogram

In [ ]:
def plot_orientations(path, nameR, nameW, f): 
    ### FINDING NORMALISING FACTOR ###
    
    #filtered normalising csv for roi
    roi1 = pd.read_csv(path+'/roi/0'+nameR, delimiter=',') 
    roi1['Major adjusted']=roi1['Major']
    roi1 = roi1[roi1['Major adjusted'] >= f]
     
    #filtered normalising csv for whole image
    whole1 = pd.read_csv(path+'/whole/0'+nameW, delimiter=',')
    whole1['Major adjusted']= whole1['Major']
    whole1= whole1[whole1['Major adjusted'] >= f] 
    
    #Normalising by frequency of first image
    bins = np.arange(0, 180,2)
    bin1, count1 = np.histogram(roi1['Angle'], bins=bins)
    min_roi_luke = np.amin(np.array(bin1)[bin1 != np.amin(bin1)])  #NORMALISER ROI
    bin2, count2 = np.histogram(whole1['Angle'], bins=bins)
    min_whole_luke = np.amin(np.array(bin2)[bin2 != np.amin(bin2)]) #NORMALISER WHOLE IMAGE
    bin3, count3 = np.abs(bin2-bin1), np.abs(count2-count1)
    min_outside = np.amin(np.array(bin3)[bin3 != np.amin(bin3)]) #NORMALISER OUTSIDE ROI

    ### PLOTTING ####
    #fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(22,9))
    bottom, top = 0, 1
    left, right = 0, 0.9
    fig.subplots_adjust(top = top, bottom = bottom, left = left, right = right, hspace = 0.3, wspace = 0.1)
    ######################################################################################################################
    select = list(range(0,37,1))#list(range(0,37,1)) #####################################################################
    colors = plt.cm.viridis(np.linspace(0,1,37))
    cmap = plt.get_cmap('viridis', 37)
    stackedL1 = np.zeros(89)#np.zeros(89)
    stackedL2 = np.zeros(89)
    
    for i in select:
        df1 = pd.read_csv(path+'/shear_band/'+str(i)+nameR, delimiter=',')
        df2 = pd.read_csv(path+'/whole_image/'+str(i)+nameW, delimiter=',')              
        df1['Major adjusted']=df1['Major']
        df2['Major adjusted']=df2['Major']
        ##Filter length below 10 pixels
        df1 = df1[df1['Major adjusted'] >= f]
        df2 = df2[df2['Major adjusted'] >= f]
        
        bins1, counts1 = np.histogram(df1['Angle'], bins)
        bins2, counts2 = np.histogram(df2['Angle'], bins)
        #histograms for each image
        ax[0,0].plot(counts1[:-1], bins1/min_roi_luke,'-',c=colors[i],  label=str(i))
        ax[0,1].plot(counts2[:-1], bins2/min_whole_luke,'-',c=colors[i], label=str(i))  
        ax[0,2].plot(counts2[:-1], np.abs(bins1 - bins2)/min_outside,'-',c=colors[i], label=str(i))  
        
        #stacking of the bins
        stackedL1 += bins1
        stackedL2 += bins2
        axis1 = counts1[:-1]
        axis2 = counts2[:-1] 
        
    
    #####COLOUR BAR
    norm = mpl.colors.Normalize(vmin=0, vmax=36)#Normalizer
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)# creating ScalarMappable
    sm.set_array([])
    #create colorbar axes
    cbar_ax = fig.add_axes([0.9, bottom, 0.05, top-bottom], aspect = 1)
    fig.colorbar(sm, cax = cbar_ax, ticks = np.linspace(0,36,37), orientation='vertical')
    for label in cbar_ax.yaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    cbar_ax.set_ylabel('Image Number')

    ###STACKED 
    ax[1,0].bar(axis1, stackedL1, width = 2,color='dodgerblue', edgecolor = 'black', alpha=0.8)
    ax[1,1].bar(axis2, stackedL2,width = 2, color='dodgerblue', edgecolor = 'black', alpha=0.8)
    ax[1,2].bar(axis2, np.abs(stackedL1-stackedL2),width = 2, color='dodgerblue', edgecolor = 'black', alpha=0.8)
    
   
    ###FINDING MODE, MEAN and NetTheta OF STACKED HISTOGRAM
    data = [stackedL1, stackedL2, np.abs(stackedL1-stackedL2)]
    region = ['--ROI--', '--WHOLE--', '--OUTSIDE ROI--']
    for m in range(3):
        print(region[m])
        
        #SMOOTH TREND AND PLOT IT #################################
        smooth_y = savgol_filter(data[m],51, 4) # window size , polynomial order 
        ax[1,m].plot(axis1, smooth_y, lw=3, c='darkorange',  label = 'Smoothed Distribution')
        #find the argmax of the smoothed trend between above 45 degrees to avoid picking 0 angle
        ims = (np.argmax(smooth_y[20:]) + 20 ) *2
        print( 'IMS = '+str(ims))
        #plot peak 
        ax[1,m].axvline(x = ims, lw=4, c='darkorange', ls='--', label = 'Smoothed Distribution Mode')
        
        #INVESTIGATE ASYMMETRY
        mid = int(len(axis1)/2) # middle INDEX of the histogram range 
        #print(axis1[mid+1], axis1[0])
        dif_sq = 0 # difference squared
        for a in range(mid+1):
            #print(a)
            dif_sq = dif_sq +((data[m][a] - data[m][88 - a]))
        MSD = dif_sq/mid # calculate the mean-sqare difference                
      
        ###CALCULATING THE MODE AND MEAN FOR EACH QUADRANT
        sel = [0, mid, len(axis1), 0]
        #print(int(len(axis1)/2))
        means = []
        modes = []
        stds = []
        for i in range(3):
            #MODE
            a = sel[i]
            b = sel[i+1]
            #print(i, a,b )
            if i == 2:
                #print('bbb')
                a = 0
                b = len(axis1)
            #print(i, a,b )
                
            max_freq_bin = np.argmax(data[m][a:b])
            
            mode_bin_edges = (axis1[max_freq_bin], axis1[max_freq_bin + 1])
            mode_value = np.mean(mode_bin_edges)
            #print('Mode '+region[m]+' = '+str(mode_value))
            if i ==1 :
                mode_value = mode_value + (int(len(axis1)/2)+1)*2
            #ax[1,m].axvline(mode_value,c='k', lw=4)
            modes.append(mode_value)
            ####MEAN WEIGHTED
            bin_centers = 0.5*(counts1[1:]+counts1[:-1])
            print(bin_centers)
            #weighted_sum = np.sum(bin_centers[a:b] * data[m][a:b])
            #total_sum = np.sum(data[m][a:b])
            #mean = weighted_sum / total_sumn #weighted
            mean =  np.mean(bin_centers) #non-weighted
            means.append(mean)
            ####STANDARD DEVIATION WEIGHTED
            #std = np.sqrt(np.cov(bin_centers[a:b]-mean, fweights= data[m][a:b]))   
            std = np.sqrt(np.cov(bin_centers[a:b]-mean))
            stds.append(std)
        
        print('Means [synthetic, antithetic, whole] = '+str(means))
        print('Stds [synthetic, antithetic, whole] = '+str(stds))
        print('Modes [synthetic, antithetic, whole] = '+str(modes))
        
        
        
        #PLOT MODES AND MEAN for legend
        ax[1,m].axvline(modes[2],c='k', ls = '--', lw=4, label = 'Mode')
        ax[1,m].axvline(means[2],c='k', ls='-', lw=4, label = 'Mean')
        
   
        ###CALCULATING THE NET THETA FOR BOTH QUADRANTS
        #first quadrant (antithetic)
        sinT_sum_1 = 0.0
        cosT_sum_1 = 0.0
       
        #second quadrant (synthetic)
        sinT_sum_2 = 0.0
        cosT_sum_2 = 0.0
       
        for b in range(0, int(len(axis1)/2)):
            #Angles
            sinT_i = data[m][b]*np.sin(np.radians(axis1[b]))
            sinT_sum_1 = sinT_sum_1 + sinT_i
            
            cosT_i = data[m][b]*np.cos(np.radians(axis1[b]))
            cosT_sum_1 = cosT_sum_1 + cosT_i
        
        for c in range(int(len(axis1)/2), len(axis1)):
            #Angles
            sinT_i = data[m][c]*np.sin(np.radians(axis1[c]))
            sinT_sum_2 = sinT_sum_2 + sinT_i
            
            cosT_i = data[m][c]*np.cos(np.radians(axis1[c]))
            cosT_sum_2 = cosT_sum_2 + cosT_i
            
    
        theta_net_1 = np.degrees(np.arctan(sinT_sum_1/cosT_sum_1))
        theta_net_2 = 180+np.degrees(np.arctan(sinT_sum_2/cosT_sum_2))
        ax[1,m].axvline(theta_net_1,c='k', lw=6, ls = ':')
        ax[1,m].text(theta_net_1 - 18, np.max(smooth_y[20:])*1.1, r'$\theta_{ns}$', size=24)
        ax[1,m].axvline(theta_net_2,c='k', lw=6,  ls = ':', label = r'$\theta_{n}$')
        ax[1,m].text(theta_net_2 + 6 ,  np.max(smooth_y[20:])*1.1, r'$\theta_{na}$', size=24)
        print('Synthetic Net Theta = '+str(theta_net_1)+', Antithetic Net Theta = '+str(theta_net_2))
      
        #measure of symmetry:
        #print('Theta_s - mode = '+str(theta_net_1-modes[2]))
        #print('Theta_a - mode = '+str(theta_net_2-modes[2]))
        Tn_syn = np.abs(theta_net_1-ims) # skewness of theta net synthetic
        Tn_ant = np.abs(theta_net_2-ims) # skewness of theta net antithetic
        num = Tn_syn - Tn_ant
        deno = Tn_syn + Tn_ant
        print('Asymmetry = '+str(num/deno))

    ###LEGEND
    ax[1,1].legend(loc="lower left", bbox_to_anchor = (-0.9, -0.55), ncol=5)
   
    ###LABELS SUBPLOTS and GRID and SCIENTIFIC NOTATION
    for n, a in enumerate(ax.flat):
        a.text(-0.08, 1.1, string.ascii_lowercase[n]+")", transform = a.transAxes, size=22, weight = 'bold')
        a.grid()
        a.ticklabel_format(style='sci', axis = 'y', scilimits = (0,0))
        a.set_xlabel(r'Orientation, $\Theta_i (^o)$')
    ###TITLES 
    ax[0,0].set_title('Shear Band')
    ax[0,1].set_title('Whole Image')
    ax[0,2].set_title('Outside Shear Band')
    
    #LABELS
    ax[0,0].set_ylabel(r'$\frac{F(\Theta_{i})}{min(F(\Theta_{0}))}$', fontsize=35)
    ax[1,0].set_ylabel(r'$\sum F(\Theta_{i})$')
    
     
   
    plt.show()

In [ ]:
# data Residual Thresholding
path_or_G = 'Residual_Thresholding_Fiji_output'
nameR_or_G = '-nt_s2_roi.csv'
nameW_or_G = '-nt_s2.csv'

# data Skeletonisation
path_or_S = 'Skeletonisation_Fiji_output'
nameR_or_S = '-s1_lukeout_roi.csv'
nameW_or_S = '-s1_lukeout.csv'

### 1. Plotting Residual Thresholding

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(22,9))
plot_orientations(path_or_G, nameR_or_G, nameW_or_G, 5)

### 2. Plotting Skeletonisation

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(22,9))
plot_orientations(path_or_S, nameR_or_S, nameW_or_S, 5)